<a href="https://colab.research.google.com/github/tayfununal/Uniform-Autoencoder-with-Latent-Flow-Matching/blob/main/ty_uae_gift/spiral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!git clone https://github.com/tayfununal/Uniform-Autoencoder-with-Latent-Flow-Matching.git

fatal: destination path 'Uniform-Autoencoder-with-Latent-Flow-Matching' already exists and is not an empty directory.


In [56]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd())

%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/models/spiral_model.ipynb
%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/datasets/spiral_dataset.ipynb

plt.rcParams['font.size'] = 20
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']


import imageio

/content


In [57]:
class Trainer:
    def __init__(self, model, optimizer, device='cpu', max_patience=20):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.device = device

        self.max_patience = max_patience
        self.best_val_loss = float('inf')
        self.patience = 0

        self.val_cost = []
        self.train_cost = []

        self.output_dir = 'latent_plots'
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
        self.image_files = []

    def train(self, train_loader, val_loader=None, epochs=10, print_every=1, name=None):
        self.model.train()

        # Eğitim başlamadan önce bir örnek görsel al (Başlangıç durumu)
        self._save_latent_scatter(0, test_loader) # İstediğin görselleştirme burada

        for epoch in range(1, epochs + 1):
            total_loss = 0.0


            for x, x_, y in train_loader:
                x = x.to(self.device)
                x_ = x_.to(self.device)

                with torch.no_grad():
                    z_hat_ = self.model.encoder(x_)
                z_hat, x_hat = self.model(x)

                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,3))

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            # Validation
            if val_loader:
                val_loss = self.validate(val_loader)

                # Early stopping check
                if val_loss < self.best_val_loss:
                    print('saved!')
                    torch.save(self.model, name + '.model')
                    self.best_val_loss = val_loss
                    self.patience = 0

                    # Her epoch sonunda görseli kaydet
                    self._save_latent_scatter(epoch, test_loader) # İstediğin görselleştirme burada

                else:
                    self.patience = self.patience + 1

                if self.patience > self.max_patience:
                    break

            if epoch % print_every == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {total_loss / len(train_loader):.6f} | Validation Loss: {val_loss:.6f}")

            self.val_cost.append(val_loss)
            self.train_cost.append(total_loss / len(train_loader))

        self.create_gif(name)

    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0.0

        with torch.no_grad():
            for x, x_, _ in val_loader:
                x = x.to(self.device)
                x_ = x_.to(self.device)

                z_hat_ = self.model.encoder(x_)
                z_hat, x_hat = self.model(x)

                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,3))

                total_loss += loss.item()

        avg_loss = total_loss / len(val_loader)
        self.model.train()
        return avg_loss

    def _save_latent_scatter(self, epoch, loader):
        """Verdiğin scatter plot mantığını kullanarak latent uzayını kaydeder."""
        self.model.eval()
        with torch.no_grad():
            # Görselleştirme için bir batch al
            X_test, y_test = next(iter(loader))
            X_test = X_test.to(self.device)

            # Modelden latent (z) değerlerini al
            z, _ = self.model(X_test)
            z = z.cpu().numpy()
            y_test = y_test.cpu().numpy()

            plt.figure(figsize=(8, 6))
            colors = ['blue', 'green', 'orange', 'red']
            labels = ['0', '1', '2', '3']

            for i in range(4):
                mask = (y_test == i)
                plt.scatter(z[mask, 0], z[mask, 1],
                            color=colors[i], label=labels[i], alpha=1.0)

            plt.xlabel('$z_1$')
            plt.ylabel('$z_2$')
            plt.title(f"Spiral in 2D Latent Space")
            #plt.legend()
            plt.grid(True, linestyle='--', alpha=1.0)
            plt.tight_layout()

            file_path = os.path.join(self.output_dir, f"epoch_{epoch:03d}.png")
            plt.savefig(file_path)
            plt.close()
            self.image_files.append(file_path)
        self.model.train()

    def create_gif(self, name):
        gif_path = f"{name}_latent_evolution.gif"
        images = [imageio.imread(f) for f in self.image_files]
        imageio.mimsave(gif_path, images, fps=5)
        print(f"GIF oluşturuldu: {gif_path}")

In [58]:

# Custom Transform
class NoiseTransform:
    """Add some noise."""

    def __init__(self, split_ratio=0.001, dim=3):

        self.split_ratio = split_ratio
        self.dim = dim

    def __call__(self, x):
      return x + self.split_ratio * torch.randn(self.dim, device='cuda' if torch.cuda.is_available() else 'cpu' , dtype=torch.float)

In [59]:
# Hyper-Parameters & Settings

dataset_size = 5000
batch_size = 250
lr = 0.001

epochs = 5000
max_patience = 5000

split_ratio = 0.0001

In [60]:
# Dataset
train_dataset = SpiralDataset(mode='train', n_samples=dataset_size, transform=NoiseTransform(split_ratio))
val_dataset = SpiralDataset(mode='val', n_samples=dataset_size, transform=NoiseTransform(split_ratio))
test_dataset = SpiralDataset(mode='test', n_samples=2*10000)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [61]:
# Create the "results" folder
os.makedirs("/content/Uniform-Autoencoder-with-Latent-Flow-Matching/results/spiral", exist_ok=True)

In [62]:
# Model
name = '/content/Uniform-Autoencoder-with-Latent-Flow-Matching/results/spiral/UAE_Spiral'
model = To_Uniform(
                 encoder_layers=[3, 128, 128, 128, 2],
                 decoder_layers=[2, 128, 128, 128, 3],
                 encoder_act=nn.ReLU,
                 decoder_act=nn.ReLU,
                 final_encoder_act=nn.Sigmoid,
                 final_decoder_act=nn.Sigmoid,
                 use_batchnorm=True
)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [63]:
# Training
trainer = Trainer(model, optimizer, device='cuda' if torch.cuda.is_available() else 'cpu', max_patience=max_patience)
trainer.train(train_loader, val_loader, epochs=epochs, name=name)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
saved!
Epoch  21 | Train Loss: 0.000435 | Validation Loss: 0.000366
Epoch  22 | Train Loss: 0.000404 | Validation Loss: 0.000482
Epoch  23 | Train Loss: 0.000470 | Validation Loss: 0.000551
Epoch  24 | Train Loss: 0.000533 | Validation Loss: 0.000422
Epoch  25 | Train Loss: 0.000452 | Validation Loss: 0.000529
saved!
Epoch  26 | Train Loss: 0.000557 | Validation Loss: 0.000363
Epoch  27 | Train Loss: 0.000488 | Validation Loss: 0.000556
Epoch  28 | Train Loss: 0.000460 | Validation Loss: 0.000375
Epoch  29 | Train Loss: 0.000413 | Validation Loss: 0.000372
Epoch  30 | Train Loss: 0.000407 | Validation Loss: 0.000411
Epoch  31 | Train Loss: 0.000475 | Validation Loss: 0.000382
Epoch  32 | Train Loss: 0.000422 | Validation Loss: 0.000384
Epoch  33 | Train Loss: 0.000492 | Validation Loss: 0.000476
Epoch  34 | Train Loss: 0.000447 | Validation Loss: 0.000364
saved!
Epoch  35 | Train Loss: 0.000424 | Validation Loss: 0.000349
Epoch  36 | Train

/tmp/ipython-input-4288835061.py:130: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(f) for f in self.image_files]


GIF oluşturuldu: /content/Uniform-Autoencoder-with-Latent-Flow-Matching/results/spiral/UAE_Spiral_latent_evolution.gif


In [64]:
# Save to CSV file
train_losses = trainer.train_cost
val_losses = trainer.val_cost

np.savetxt("/content/Uniform-Autoencoder-with-Latent-Flow-Matching/results/spiral/losses.csv",
           np.column_stack((train_losses, val_losses)),
           delimiter=",",
           header="train_loss,val_loss",
           comments="")

In [65]:
rm -r /content/latent_plots